# Training example

# Prepare

## 1 - Load modules

In [ ]:
# make sure plot shows
%matplotlib inline
import matplotlib.pyplot as plt
from lfv_pdnn.common import array_utils
from lfv_pdnn.data_io import get_arrays
from lfv_pdnn.train import model, train_utils

## 1 - Define constants & functions for training.

In [ ]:
selected_features = ["m_ll", "ele_pt", "ele_istight", "mu_pt", "mu_istight", "pt_ll", "dr_ll", "met", "njets"]
plot_bkg_list = ["wjets", "diboson", "zll", "top"]
input_path = "/data/arrays/rel_103"

## 2 - Load & modify arrays

In [ ]:
sig_list = ["rpv_500", "rpv_700", "rpv_1000", "rpv_1500", "rpv_2000"]
sig_dict = get_arrays.get_sig(input_path, "run2", "emu", sig_list, selected_features)

bkg_list = ["diboson", "top", "wjets", "zll"]
bkg_dict = get_arrays.get_bkg(input_path, "run2", "emu", bkg_list, selected_features)


## 3 - Example for using the code

In [ ]:
# training example
model_deep = model.Model_1016(
    "model_test",
    len(selected_features),
    learn_rate=0.005,
    decay=1e-4,
    selected_features=selected_features)
xs = array_utils.modify_array(sig_dict["all_norm"], select_channel=True, remove_negative_weight=True)
xb = array_utils.modify_array(bkg_dict["all"], select_channel=True, remove_negative_weight=True)
model_deep.prepare_array(xs, xb, reset_mass=True, reset_mass_name="m_ll")
model_deep.compile()
model_deep.train(batch_size=64, epochs=20, val_split=0.2, sig_class_weight=1., bkg_class_weight=1.)
model_deep.save_model(save_dir=".", file_name="model_test")

In [ ]:
# Example for showing performance
model_deep.show_performance(figsize=(16, 18))
fig, ax = plt.subplots(ncols=2, figsize=(16, 4))
model_deep.plot_scores_separate(ax[0], bkg_dict, plot_bkg_list, selected_features,
                                sig_arr=None, sig_weights=None, plot_title='training scores',
                                bins=40, range=(-0.25, 1.25), density=True, log=False)
model_deep.plot_scores_separate(ax[1], bkg_dict, plot_bkg_list, selected_features,
                                sig_arr=None, sig_weights=None, plot_title='training scores',
                                bins=40, range=(-0.25, 1.25), density=True, log=True)

In [ ]:
# Example for loading model and make plots
xs = array_utils.modify_array(sig_dict["rpv_500"], select_channel=True, remove_negative_weight=True)
xb = array_utils.modify_array(bkg_dict["all"], select_channel=True, remove_negative_weight=True)
model_load = model.Model_1016(
    "model_load_test",
    len(selected_features),
    learn_rate=0.005,
    decay=1e-4,
    selected_features=selected_features)
model_load.prepare_array(xs, xb, reset_mass=False)
model_load.load_model('model_test', dir=".")
model_load.show_performance(figsize=(16, 18))
fig, ax = plt.subplots(ncols=2, figsize=(16, 4))
model_load.plot_scores_separate(ax[0], bkg_dict, plot_bkg_list, selected_features,
                                sig_arr=None, sig_weights=None, plot_title='training scores',
                                bins=40, range=(-0.25, 1.25), density=True, log=False)
model_load.plot_scores_separate(ax[1], bkg_dict, plot_bkg_list, selected_features,
                                sig_arr=None, sig_weights=None, plot_title='training scores',
                                bins=40, range=(-0.25, 1.25), density=True, log=True)